<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/%EC%B5%9C%EC%A0%95%EC%9A%B1/llm_2024_Lab04_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4. LoRA

Updated by geonho lee 2024.8.29

Created by minsoo kim 2023.11.15

# Part 0. Install Package & Import ⚒️

In [ ]:
#!pip install -U bitsandbytes
#!pip install -U git+https://github.com/huggingface/transformers.git
!pip install transformers
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-e87rf6n7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-e87rf6n7
  Resolved https://github.com/huggingface/peft.git to commit 679bcd8777fc8215208bc46b7f54f1f4061791ae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
import datasets, torch

# Part 1. LLM Fine-Tuning with LoRA (Basic) 🚀

###  Step 1. Load AutoModel (OPT-2.7B)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "facebook/opt-2.7b"
# Model Load
model_opt = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto", cache_dir="/home/ms/hf_cache/hub")
# Tokenizer Load
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/home/ms/hf_cache")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
print(model_opt)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2560, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2560)
      (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (final_layer_norm): Laye

In [ ]:
print(model_opt.model.decoder.layers[0].self_attn.q_proj.weight)
print(f"requires_grad: {model_opt.model.decoder.layers[0].self_attn.q_proj.weight.requires_grad}")

Parameter containing:
tensor([[ 0.0109,  0.0019,  0.0061,  ..., -0.0097,  0.0081, -0.0143],
        [ 0.0122,  0.0041,  0.0047,  ..., -0.0040, -0.0068, -0.0022],
        [ 0.0103,  0.0122, -0.0070,  ..., -0.0005,  0.0052,  0.0169],
        ...,
        [-0.0063,  0.0079,  0.0016,  ..., -0.0271, -0.0070, -0.0066],
        [-0.0230,  0.0130, -0.0002,  ...,  0.0053,  0.0009, -0.0083],
        [ 0.0117, -0.0104,  0.0215,  ..., -0.0037, -0.0172, -0.0024]],
       device='cuda:0', dtype=torch.float16, requires_grad=True)
requires_grad: True


In [ ]:
print(model_opt.lm_head.weight)
print(f"requires_grad: {model_opt.lm_head.weight.requires_grad}")

Parameter containing:
tensor([[ 2.9190e-02,  3.1250e-02,  1.0704e-02,  ...,  2.0844e-02,
          7.9422e-03, -2.9449e-02],
        [ 2.9175e-02,  3.1250e-02,  1.0101e-02,  ...,  2.0065e-02,
          7.9117e-03, -2.9755e-02],
        [-1.4565e-02, -2.7039e-02, -3.7231e-03,  ..., -2.3102e-02,
         -5.6088e-05, -4.3488e-03],
        ...,
        [ 2.8778e-02,  3.1250e-02,  1.0590e-02,  ...,  1.9196e-02,
          7.9193e-03, -2.9099e-02],
        [ 2.8488e-02,  3.1250e-02,  1.1093e-02,  ...,  1.9318e-02,
          7.8888e-03, -2.9602e-02],
        [ 2.8976e-02,  3.1250e-02,  1.0414e-02,  ...,  1.9470e-02,
          7.9041e-03, -2.9770e-02]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
requires_grad: True


In [ ]:
print(model_opt.model.decoder.embed_tokens.weight)
print(f"requires_grad: {model_opt.model.decoder.embed_tokens.weight.requires_grad}")

Parameter containing:
tensor([[ 2.9190e-02,  3.1250e-02,  1.0704e-02,  ...,  2.0844e-02,
          7.9422e-03, -2.9449e-02],
        [ 2.9175e-02,  3.1250e-02,  1.0101e-02,  ...,  2.0065e-02,
          7.9117e-03, -2.9755e-02],
        [-1.4565e-02, -2.7039e-02, -3.7231e-03,  ..., -2.3102e-02,
         -5.6088e-05, -4.3488e-03],
        ...,
        [ 2.8778e-02,  3.1250e-02,  1.0590e-02,  ...,  1.9196e-02,
          7.9193e-03, -2.9099e-02],
        [ 2.8488e-02,  3.1250e-02,  1.1093e-02,  ...,  1.9318e-02,
          7.8888e-03, -2.9602e-02],
        [ 2.8976e-02,  3.1250e-02,  1.0414e-02,  ...,  1.9470e-02,
          7.9041e-03, -2.9770e-02]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
requires_grad: True


In [ ]:
print(model_opt.model.decoder.layers[0].self_attn_layer_norm.weight)
print(f"requires_grad: {model_opt.model.decoder.layers[0].self_attn_layer_norm.weight.requires_grad}")

Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0', dtype=torch.float16,
       requires_grad=True)
requires_grad: True


In [ ]:
print_trainable_parameters(model_opt)

trainable params: 2651596800 || all params: 2651596800 || trainable%: 100.0


### Step 2. LoRA Implementation (PEFT package)

In [ ]:
from peft import LoraConfig, get_peft_model
# Setting for LoRA PEFT (fine-tuning QKV projection weight)
config = LoraConfig(
    r=4, # LoRA rank [2,4,8,16,64,...]
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"], # target modules ["fc1", "fc2", "q_proj", "k_proj", "v_proj"]
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to OPT Pre-Trained Model
model_opt.gradient_checkpointing_enable()
model_opt = get_peft_model(model_opt, config)
print_trainable_parameters(model_opt)

trainable params: 1966080 || all params: 2653562880 || trainable%: 0.07409208256636451


In [ ]:
model_opt

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 2560, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 2560)
          (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0-31): 32 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2560, out_features=4, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=4, out_features=2560, bias=False)
         

In [ ]:
model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj

lora.Linear(
  (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=2560, out_features=4, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=4, out_features=2560, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)

In [ ]:
print(model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj.base_layer)
print(model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj.base_layer.weight)

Linear(in_features=2560, out_features=2560, bias=True)
Parameter containing:
tensor([[ 0.0094,  0.0021, -0.0069,  ...,  0.0093, -0.0093,  0.0117],
        [ 0.0054, -0.0086, -0.0053,  ..., -0.0078,  0.0078, -0.0085],
        [ 0.0072, -0.0038, -0.0041,  ..., -0.0133,  0.0115,  0.0033],
        ...,
        [-0.0323,  0.0282, -0.0052,  ...,  0.0138,  0.0205, -0.0055],
        [ 0.0165, -0.0093,  0.0135,  ...,  0.0112, -0.0135,  0.0220],
        [ 0.0103,  0.0037, -0.0058,  ...,  0.0133,  0.0157,  0.0367]],
       device='cuda:0', dtype=torch.float16)


In [ ]:
print(model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj.lora_A)
print(model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj.lora_A.default.weight)

ModuleDict(
  (default): Linear(in_features=2560, out_features=4, bias=False)
)
Parameter containing:
tensor([[ 0.0143,  0.0168, -0.0045,  ...,  0.0121,  0.0094, -0.0117],
        [-0.0117, -0.0056,  0.0157,  ..., -0.0150,  0.0092, -0.0067],
        [-0.0032, -0.0018, -0.0058,  ...,  0.0132,  0.0117, -0.0106],
        [ 0.0110, -0.0014, -0.0152,  ...,  0.0094,  0.0064,  0.0052]],
       device='cuda:0', requires_grad=True)


In [ ]:
print(model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj.lora_B)
print(model_opt.base_model.model.model.decoder.layers[0].self_attn.k_proj.lora_B.default.weight)

ModuleDict(
  (default): Linear(in_features=4, out_features=2560, bias=False)
)
Parameter containing:
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0', requires_grad=True)


### Step 3. Pre-Precess & Tokenize Instruction Dataset

In [ ]:
from datasets import load_dataset

data = load_dataset("piqa", split="train[:10%]").select(range(100)) # sample 100 data from PIQA train dataset
column_names = data.column_names

The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})

In [ ]:
column_names

['goal', 'sol1', 'sol2', 'label']

In [ ]:
data["goal"]

["When boiling butter, when it's ready, you can",
 'To permanently attach metal legs to a chair, you can',
 'how do you indent something?',
 'how do you shake something?',
 'Clean tires',
 'how do you taste something?',
 'To create a makeshift ice pack,',
 "What should I use as a stain on a wooden bowl I've just made.",
 'How to boil eggs.',
 'how do you stab something?',
 'How to quickly soften butter for baking?',
 'How can I save my milk if I am going to be leaving town for a bit?',
 'how do you open a capri-sun',
 'how do you turn on an ipad screen?',
 'How to season raw chicken.',
 'pin',
 'How to tie pieces of paper together.',
 'how do you get leg muscles?',
 'To make a tomato sauce taste more like a pizza sauce,',
 'how do you draw with chalk?',
 'Convert unused paint to solid waste.',
 'How to check your Facebook feed',
 'How to Prepare Puffed Pastry for Baked Brie Pumpkin',
 'how do I make chocolate pasta?',
 'how do you flood a room?',
 'How else can you mix the biscuits and

Pre-Processing (concat goal and solution)

In [ ]:
def add_sol_with_label(example):
  sentence = example[column_names[0]] + " "
  answer = example[column_names[1]] if example["label"] == 0 else example[column_names[2]]

  example["sentence"] = sentence + answer
  return example

In [ ]:
# Pre-Processing PIQA train dataset
updated_data = data.map(add_sol_with_label)
updated_data = updated_data.remove_columns("goal")
updated_data = updated_data.remove_columns("label")
updated_data = updated_data.rename_column("sentence", "goal")
data = updated_data

# Tokenize
data = data.map(lambda samples:tokenizer(samples["goal"]), batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# data["goal"]

In [ ]:
# Check pre-processed training dataset
print(data["goal"][2])
print(data["input_ids"][2])

how do you indent something? leave a space before starting the writing
[2, 9178, 109, 47, 42959, 402, 116, 989, 10, 980, 137, 1158, 5, 2410]


###  Step 4. Text Generation before Fine-Tuning

In [ ]:
# Insert any context text you want
text = "When boiling butter, "
device = "cuda:0"

# Set max sequence length
max_token_number = 20

# Tokenize input sequence
inputs = tokenizer(text, return_tensors="pt").to(device)

# Text generation (model inference)
with torch.no_grad():
    outputs_opt = model_opt.generate(**inputs, max_new_tokens=max_token_number)

print(tokenizer.decode(outputs_opt[0], skip_special_tokens=True))

When boiling butter,  do you use a pot or a pan?
I use a pot. I don't have a


### Step 5. HuggingFace Trainer Setting for LLM Fine-Tuning

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

# Fine-Tuning Setting
trainer = transformers.Trainer(
    model=model_opt,
    train_dataset=data,
     args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        max_steps=100,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model_opt.config.use_cache = False  # silence the warnings. Please re-enable for inference!


max_steps is given, it will override any value given in num_train_epochs


### Let's Fine-Tuning 🔥🔥🔥

In [ ]:
# 100 step Fine-Tuning
trainer.train()

Step,Training Loss
10,3.480300
20,3.151700
30,2.929700
40,3.739800
50,3.087200
60,3.331700
70,3.345100
80,2.979400
90,2.749000
100,2.844000


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

TrainOutput(global_step=100, training_loss=3.1637869262695313, metrics={'train_runtime': 29.1173, 'train_samples_per_second': 3.434, 'train_steps_per_second': 3.434, 'total_flos': 48799972270080.0, 'train_loss': 3.1637869262695313, 'epoch': 1.0})

### Step 6. Text Generation after Fine-Tuning 📚

In [ ]:
# Insert any context text you want
text = "When boiling butter, "
device = "cuda:0"

# Set max sequence length
max_token_number = 30

# Tokenize input sequence
inputs = tokenizer(text, return_tensors="pt").to(device)

# Text generation (model inference)
with torch.no_grad():
    outputs_opt = model_opt.generate(**inputs, max_new_tokens=max_token_number)

print(tokenizer.decode(outputs_opt[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


When boiling butter,  put it in a pot and cover with water.  Let it boil for a few minutes.  Then turn off the heat and let it sit for


# Part 2. Fine-Tuning 🇰🇷 LLM with LoRA 🚀🚀

### Step1. Load Korean Instruction Dataset (KoAlpaca)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

In [ ]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
# Pre-Processing Korean Instruction dataset
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
# Sample Korean Instruction dataset
print(data["train"]["text"][1])

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

### 질문: 스웨터의 유래는 어디에서 시작되었나요?

### 답변: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다.<|endoftext|>


### Step2. Load Korean Pre-Trained LLM

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "EleutherAI/polyglot-ko-1.3b"
# Model Load
model_ko = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto", cache_dir="/home/ms/hf_cache/hub")
# Tokenizer Load
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/home/ms/hf_cache")

config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/748M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Step3. Text Generation before Fine-Tuning

In [ ]:
model_ko.eval()
model_ko.config.use_cache = True  # silence the warnings. Please re-enable for inference!
# Insert any context text you want
question = "건강하게 살려면 어떻게 해야하나요?"
text = f"### 질문: {question}\n\n### 답변:"
device = "cuda:0"

# Set max sequence length
max_token_number = 80

# Tokenize input sequence
inputs = tokenizer(text, return_tensors="pt", return_token_type_ids=False).to(device)

# Text generation (model inference)
with torch.no_grad():
    outputs_ko = model_ko.generate(**inputs, max_new_tokens=max_token_number)

print(tokenizer.decode(outputs_ko[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살려면 어떻게 해야하나요?

### 답변: 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요? 건강하게 살려면 어떻게 해야하나요?


### Step4. Attach LoRA adapter

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_ko.gradient_checkpointing_enable()
model_ko = get_peft_model(model_ko, config)
print_trainable_parameters(model_ko)

trainable params: 1572864 || all params: 1333383168 || trainable%: 0.11796039111242178


### Step5. Fine-Tuning 🔥🔥🔥

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.7 MB/s eta 0:00:00


In [ ]:
import transformers

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

model_ko.train()
# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model_ko,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=200,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model_ko.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,2.721000
20,2.451300
30,2.446200
40,2.344700
50,2.371800
60,2.171900
70,2.175400
80,2.209700
90,2.165000
100,2.238800


TrainOutput(global_step=200, training_loss=2.2056573581695558, metrics={'train_runtime': 78.2578, 'train_samples_per_second': 5.111, 'train_steps_per_second': 2.556, 'total_flos': 866768483205120.0, 'train_loss': 2.2056573581695558, 'epoch': 0.018907165815844205})

### Step6. Text Generation after Fine-Tuning

In [ ]:
model_ko.eval()
model_ko.config.use_cache = True  # silence the warnings. Please re-enable for inference!
# Insert any context text you want
question = "건강하게 살려면 어떻게 해야하나요?"
text = f"### 질문: {question}\n\n### 답변:"
device = "cuda:0"

# Set max sequence length
max_token_number = 50

# Tokenize input sequence
inputs = tokenizer(text, return_tensors="pt", return_token_type_ids=False).to(device)

# Text generation (model inference)
with torch.no_grad():
    outputs_ko = model_ko.generate(**inputs, max_new_tokens=max_token_number)
print(tokenizer.decode(outputs_ko[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살려면 어떻게 해야하나요?

### 답변: 건강하게 살려면, 우선 건강한 식습관을 가지는 것이 중요합니다. 건강한 식습관은 건강한 신체를 유지하는 데 도움이 되며, 건강한 신체는 건강한 정신을 유지하는 데 도움이 됩니다


In [ ]:
model_ko

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): GPTNeoXForCausalLM(
          (gpt_neox): GPTNeoXModel(
            (embed_in): Embedding(30080, 2048)
            (emb_dropout): Dropout(p=0.0, inplace=False)
            (layers): ModuleList(
              (0-23): 24 x GPTNeoXLayer(
                (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
                (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
                (post_attention_dropout): Dropout(p=0.0, inplace=False)
                (post_mlp_dropout): Dropout(p=0.0, inplace=False)
                (attention): GPTNeoXSdpaAttention(
                  (rotary_emb): GPTNeoXRotaryEmbedding()
                  (query_key_value): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=6144, bias=True)
                    (lora_dropout): ModuleDict(
           

In [ ]:
model_ko = get_peft_model(model_ko, config, adapter_name="ai_expert")

In [ ]:
model_ko

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): GPTNeoXForCausalLM(
              (gpt_neox): GPTNeoXModel(
                (embed_in): Embedding(30080, 2048)
                (emb_dropout): Dropout(p=0.0, inplace=False)
                (layers): ModuleList(
                  (0-23): 24 x GPTNeoXLayer(
                    (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
                    (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
                    (post_attention_dropout): Dropout(p=0.0, inplace=False)
                    (post_mlp_dropout): Dropout(p=0.0, inplace=False)
                    (attention): GPTNeoXSdpaAttention(
                      (rotary_emb): GPTNeoXRotaryEmbedding()
                      (query_key_value): lora.Linear(
                      

In [ ]:
model_ko.base_model.model.base_model.model.base_model.model.gpt_neox.layers[0].attention.query_key_value.lora_B.default.weight

Parameter containing:
tensor([[-1.3114e-03, -9.3751e-04, -1.1270e-03,  ..., -1.4413e-04,
         -1.0568e-03, -6.4898e-04],
        [-8.2109e-04, -7.3814e-04,  2.9786e-04,  ...,  8.0829e-04,
         -5.7448e-05, -4.0005e-04],
        [-6.9543e-05,  6.2630e-04,  2.8945e-04,  ...,  5.5007e-04,
          1.2866e-04,  4.6992e-04],
        ...,
        [ 3.5079e-03,  2.6444e-03,  1.6324e-03,  ..., -8.9388e-04,
          4.5622e-03,  3.1500e-03],
        [ 8.8486e-04,  4.3442e-04,  6.2545e-04,  ...,  3.9554e-04,
          1.2530e-03,  1.4288e-03],
        [ 7.6235e-04,  1.4571e-03,  9.1138e-04,  ...,  1.2668e-03,
          8.7782e-04,  1.0460e-03]], device='cuda:0')

In [ ]:
model_ko.base_model.model.base_model.model.base_model.model.gpt_neox.layers[0].attention.query_key_value.lora_B.ai_expert.weight

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', requires_grad=True)